#### Imports/setup

In [1]:
from parameters.config_imports import *

In [2]:
def whisp_stats_as_df (roi): 
    %store roi
    %run process_whisp_stats.ipynb   
    return df_out

#### Inputs : feature collection (without Geo IDs)
- Choose between 1) GEE asset or 2) Geojson input
- Comment out the one you dont want

In [3]:
# asset_id = "projects/ee-andyarnellgee/assets/example_asset" # asset id
# roi = ee.FeatureCollection(asset_id) # load feature collection asset

geojson_path = "input_examples/geojson_example.geojson" # geojson file
roi = geojson_to_ee(geojson_path) # convert geojson to feature collection

In [4]:
roi # view feature collection properties

In [5]:
# optional code: uncomment below line to test converting to point inputs using polygon centroid

# roi = roi.map(lambda feature: ee.Feature(feature.geometry().centroid(1))) 

### Whisp it
- If over 500 polygons currently this will go to Google Drive and require manual uploading here
- This value can be changed in parameters.config_runtime.py 
- NB Currently stats processing sent to a separate notebook 'process_whisp_stats.pynb'

In [6]:
df = whisp_stats_as_df(roi)

Stored 'roi' (FeatureCollection)
Datasets present in lookup:  True
geo id not present ['Plot_area_ha', 'Geometry_type', 'Country', 'Admin_Level_1', 'Centroid_lon', 'Centroid_lat', 'Unit']
...stats processing started
...ee to df processing time: 7.792349338531494 seconds


### Display it


In [7]:
df

,Plot_area_ha,Geometry_type,Country,Admin_Level_1,Centroid_lon,Centroid_lat,Unit,EUFO_2020,GLAD_Primary,TMF_undist,...,GFC_loss_before_2020,ESA_fire_before_2020,MODIS_fire_before_2020,RADD_before_2020,TMF_deg_after_2020,TMF_def_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,system:index
0,1.938,Polygon,GHA,Ashanti Region,-1.611942,6.159539,ha,0.362,0.000,0.000,...,1.552,0.00,0.000,0.000,0,0,0.000,0,0.000,0
1,16.602,Polygon,GHA,Western Region,-2.157144,5.981149,ha,16.602,14.493,15.792,...,0.000,0.00,0.000,1.111,0,0,0.000,0,0.973,1
2,31.210,Polygon,IDN,South Sumatra,103.956096,-3.054668,ha,0.000,0.000,0.000,...,31.020,31.21,6.401,0.000,0,0,0.000,0,0.000,2
3,20.884,Polygon,IDN,South Sumatra,103.977512,-3.083808,ha,5.841,0.000,6.472,...,9.043,0.00,0.000,0.000,0,0,0.000,0,0.000,3
4,8.279,Polygon,CIV,Lagunes,-4.101646,5.711935,ha,0.446,0.001,0.023,...,3.194,0.00,0.000,0.000,0,0,1.602,0,0.001,4
5,3.613,Polygon,CIV,Montagnes,-7.507022,6.071468,ha,1.485,0.000,0.000,...,1.955,0.00,0.000,0.000,0,0,0.000,0,0.000,5


#### Export it to CSV file
NB if exporting to Google Drive skip this and upload to your results folder 'whisp_output_table.csv'

In [8]:
#export output csv (default: '/home/sepal-user/module_results/whisp/whisp_output_table.csv')
df.to_csv(path_or_buf=out_csv,header=True,index=False)

print ("output csv: ", out_csv)

output csv:  /home/sepal-user/module_results/whisp/whisp_output_table.csv


#### Add Geo ids (optional)
##### Step 1. Register polygons:  compiles geo_ids in a lookup csv
- NB registration takes a long time if many polygons (a few seconds each one). AgStack will update this in future. 
- If processing doesn't complete, just rerun this cell and it should pickup from where you left.
- If registering many features you can also click dollar sign in bottom right of Sepal and under 'Sessions' heading extend the length as needed.

In [ ]:
register_fc_and_append_to_csv(
    feature_col=roi,
    geo_id_column="Geo_id",
    output_lookup_csv=out_directory/"temp_geo_id_lookup.csv",
    join_id_column="system:index",
    token=token,
    session=session,
    asset_registry_base=asset_registry_base,
    override_checks=False,
    remove_temp_csv=False,
    debug=True)

Chosen path for temp lookup csv:/home/sepal-user/module_results/whisp/temp_geo_id_lookup.csv 

Number without geo ids:6
Processing started...
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d


##### Step 2. Join geo ids from lookup csv to Whisp stats csv

In [ ]:
add_geo_ids_to_csv_from_lookup_csv(
        input_csv=out_directory/"whisp_output_table.csv",
        geo_id_lookup_csv=out_directory/"temp_geo_id_lookup.csv",
        join_id_column="system:index",
        geo_id_column=geo_id_column,
        overwrite=False,
        drop_geo=False,
        debug=True)

##### Optional: remove "system:index" column 
NB this is needed for joining geo_ids to csv (from lookup table). Check you have all your geo_ids first and if in doubt run on a copy of the results


In [ ]:
# remove_column_from_csv(
#     csv_file="whisp_output_table_w_geo_id.csv", # this may change depending on if overwrite is on
#     column_name="system:index"
# )

##### Calculate EUDR risk category

In [ ]:
# Replace with name of your CSV file 
# NB if skipped 'Add Geo ids' should set to "out_directory/whisp_output_table.csv" 
csv_file_input =out_directory/'whisp_output_table_w_geo_id.csv'

# import csv as a dataframe
df_w_stats = pd.read_csv(csv_file_input)

df_w_risk = whisp_risk(
    df = df_w_stats, 
    ind_1_pcent_threshold=10,
    ind_2_pcent_threshold=10,
    ind_3_pcent_threshold=0,
    ind_4_pcent_threshold=0
    )


##### Display table with risk columns

In [ ]:
df_w_risk

##### Export table with risk columns to csv 

In [ ]:
output_file=out_directory/'whisp_output_table_w_risk.csv' # edit as required

df_w_risk.to_csv(output_file)
